In [213]:
import requests
from pprint import pprint
import requests
import time
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import pymorphy2
import time
import os
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
from tqdm import tqdm
import math
import numpy as np
import string
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as layer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from time import sleep
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split   
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
session = requests.session()

In [3]:
ua = UserAgent(verify_ssl=False)

In [4]:
# информация из страницы со списком новостей
def parse_news_page_block(one_block):
    block = {}
    ua = UserAgent(verify_ssl=False)
    a = one_block.find('a')
    block['href'] = a.attrs['href']
    return block

In [5]:
# отдельная страница новости   
def parse_one_article(block):
    ua = UserAgent(verify_ssl=False)
    url_one = 'https://spasibovsem.ru' + block['href']
    block['page_id'] = block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    time.sleep(2)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find("div", {"class": "text response-text description"}, {"itemprop": "reviewBody"}).text   
    block['rating'] = (str(soup.find("abbr", {"class": "rating"}))).split('>')[0].split()[2].split('"')[1]
    return block 


In [6]:
def get_nth_page(page_number):
    # скачиваем
    url =  f'https://spasibovsem.ru/filmy-otzyvy/?page={page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})
    time.sleep(2)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    # находим новости
    news = soup.find_all('div', {'class': 'response-name'})
    
    # идем по новостям и обрабатываем их
    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            print(e)
    seen_news = []
    # идем по отдельным статьям и достаем информацию
    result = []
    for b in blocks:
        try:
            res = parse_one_article(b)
            res['id'] = b['page_id']
            result.append(res)
        except Exception as e:
            print(e)
    
    # возвращаем найденную информацию
    return result

In [7]:
def run_all(n_pages):
    reses = []
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+20)
        reses.append(blocks)
    return reses

In [8]:
spis1 = run_all(20)

  0%|          | 0/20 [00:00<?, ?it/s]

 20%|██        | 4/20 [06:59<28:15, 105.97s/it]

Invalid return character or leading space in header: User-Agent


 30%|███       | 6/20 [11:44<29:36, 126.86s/it]

Invalid return character or leading space in header: User-Agent


 40%|████      | 8/20 [16:18<26:26, 132.24s/it]

Invalid return character or leading space in header: User-Agent


 70%|███████   | 14/20 [30:45<14:29, 144.92s/it]

Invalid return character or leading space in header: User-Agent


100%|██████████| 20/20 [45:29<00:00, 136.47s/it]


In [10]:
spis2 = run_all(30)

  0%|          | 0/30 [00:00<?, ?it/s]

Invalid return character or leading space in header: User-Agent


 20%|██        | 6/30 [09:47<39:20, 98.35s/it]

Invalid return character or leading space in header: User-Agent
Invalid return character or leading space in header: User-Agent


 27%|██▋       | 8/30 [12:56<35:14, 96.10s/it]

...

 83%|████████▎ | 25/30 [49:00<11:27, 137.51s/it]

Invalid return character or leading space in header: User-Agent


 93%|█████████▎| 28/30 [55:47<04:33, 136.50s/it]

Invalid return character or leading space in header: User-Agent


100%|██████████| 30/30 [1:00:24<00:00, 120.81s/it]


In [171]:
df = pd.DataFrame(columns =['texts', 'rating', 'tonality'])

In [170]:
clus = [spis1, spis2]
texts = []
ratings = []
for spis in clus:
    for page in spis:
        for slovik in page:
            texts.append(slovik['full_text'])
            ratings.append(slovik['rating'])

In [172]:
df['texts'] = texts
df['rating'] = ratings

In [173]:
tone_dictionary ={'1' : 0, '2' : 0, '4' : 1, '5' : 1, '3' : 0.5}
#df[df['rating']=='3']['tonality'] = 0.5
df['tonality'] = df['rating'].map(tone_dictionary)

- Токенизация, приведение к нижнему регистру и к начальной форме 
- 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (2 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)
- Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)
- Предложите как минимум 2 способа улучшить этот алгоритм определения тональности отзыва (1 балл за описание и реализацию каждого способа; если 2 способа описаны только текстом, это 1 балл. За третий и последующие способы дополнительных баллов не будет)

In [163]:
morph = MorphAnalyzer()
stops = stopwords.words("russian")

In [174]:
reviews_preprocessed = []
for a_text in list(df['texts'].values):
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form.lower() for item in a_tokens])
    reviews_preprocessed.append(a_lemmatized)

In [175]:
reviews_preprocessed

['иногда казаться , что если быть создать лекарство от неизлечимый болезнь , то конечно нужно сделать это достояние общественность , но у больший корпорация , который зарабатывать на чужой болезнь , мочь бы возникнуть проблема , о чем речь и идти в новый триллер " на прицел ". пять учёный разработать лекарство от рак , который бесплатно хотеть обнародовать , но вдруг они по один стать убивать ... актёр знакомый я здесь не быть , да и по больший часть я они игра не устроить , какой - то неестественный . по время тоже можно быть подсократить сюжет . кино оказаться не зрелищный , ну чуть можно попереживать за два выжить разработчик лекарственный формула , но как - то всё предсказуемый и простоватый снятой . преступник действовать не в одиночка , но по крайний мера один из они почти сразу можно быть заподозрить . развязка у этот история неплохой впрочем .',
 'российский комедия зачастую оказываться весьма туповатый , стараться такой обычно не смотреть , но новый фильм " рассмешить я " прив

In [176]:
slovik_pos = {}
slovik_neg = {}
for text, tone in zip(reviews_preprocessed, list(df['tonality'].values)):
    if tone == 0:
        for word in text.split():
            if word not in stops and word not in string.punctuation and word not in slovik_neg:
                slovik_neg[word] = 1
            elif word not in stops and word not in string.punctuation and word in slovik_neg:
                slovik_neg[word] += 1
    if tone == 1:
        for word in text.split():
            if word not in stops and word not in string.punctuation and word not in slovik_pos:
                slovik_pos[word] = 1
            elif word not in stops and word not in string.punctuation and word in slovik_pos:
                slovik_pos[word] += 1    

In [177]:
sloviki = [slovik_neg, slovik_pos]
for slovik in sloviki:
    for k, v in slovik.copy().items():
        if v<=2:
            del slovik[k]

In [178]:
neg_set = []
pos_set = []
for k in slovik_neg.keys():
    if k not in list(slovik_pos.keys()):
        neg_set.append(k)
for k in slovik_pos.keys():
    if k not in list(slovik_neg.keys()):
        pos_set.append(k)

In [179]:
print(len(neg_set),len(pos_set))

235 4788


In [73]:
with open("negative_set.txt", "a") as file:
    print(*neg_set, file=file)
with open("positive_set.txt", "a") as file:
    print(*pos_set, file=file)

In [152]:
#так как выборка отзывов несбалансирована (положительных отзывов гораздо больше), 
# напишем функцию, чтобы выкачать еще отзывы, и оставим только отрицательные
def run_neg(n_pages):
    reses = []
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+80)
        reses.append(blocks)
    return reses

In [153]:
spis3 = run_neg(20)

  5%|▌         | 1/20 [03:01<57:22, 181.17s/it]

Invalid return character or leading space in header: User-Agent


 40%|████      | 8/20 [22:02<32:42, 163.57s/it]

Invalid return character or leading space in header: User-Agent


 65%|██████▌   | 13/20 [35:13<18:35, 159.38s/it]

...

 85%|████████▌ | 17/20 [46:08<08:08, 162.82s/it]

Invalid return character or leading space in header: User-Agent
Invalid return character or leading space in header: User-Agent


 95%|█████████▌| 19/20 [51:45<02:46, 166.27s/it]

Invalid return character or leading space in header: User-Agent


100%|██████████| 20/20 [54:20<00:00, 163.05s/it]


In [157]:
len(spis3)
#добавляем список в списки, сформированные парсером, и по новой вызываем функции

20

In [183]:
#достаем выборку для тестирования функции, она составляет ~30% от обучающей выборки
def run_test(n_pages):
    reses = []
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+100)
        reses.append(blocks)
    return reses

In [184]:
test_spis = run_test(14)

  7%|▋         | 1/14 [02:48<36:31, 168.56s/it]

Invalid return character or leading space in header: User-Agent


 14%|█▍        | 2/14 [05:32<33:12, 166.08s/it]

Invalid return character or leading space in header: User-Agent


 43%|████▎     | 6/14 [16:21<21:42, 162.78s/it]

Invalid return character or leading space in header: User-Agent


 93%|█████████▎| 13/14 [35:03<02:41, 161.45s/it]

Invalid return character or leading space in header: User-Agent


100%|██████████| 14/14 [37:42<00:00, 161.61s/it]


In [185]:
test = pd.DataFrame(columns =['texts', 'rating', 'tonality'])

In [186]:
clus = [test_spis]
texts = []
ratings = []
for spis in clus:
    for page in spis:
        for slovik in page:
            texts.append(slovik['full_text'])
            ratings.append(slovik['rating'])

In [187]:
test['texts'] = texts
test['rating'] = ratings

In [188]:
test['tonality'] = test['rating'].map(tone_dictionary)

In [194]:
test = test.loc[test['tonality'] != 0.5]

In [196]:
reviews_test_preprocessed = []
for a_text in list(test['texts'].values):
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form.lower() for item in a_tokens])
    reviews_test_preprocessed.append(a_lemmatized)

In [197]:
res_tone = []
errors = 0
for text in reviews_test_preprocessed:
    neg = 0
    pos = 0
    for word in text.split():
        if word not in stops and word not in string.punctuation and word in neg_set:
            neg+=1
        elif word not in stops and word not in string.punctuation and word in pos_set:
            pos+=1
    if neg>pos:
        res_tone.append(0)
    elif pos>neg:
        res_tone.append(1)
    else:
        res_tone.append(0.5)
        errors+=1
print(errors)
res_tone

0


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [199]:
print(res_tone, test['tonality'])
print(sum(1 for x,y in zip(res_tone, test['tonality']) if x == y) / len(res_tone))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## Способы улучшения классификации

Для улучшения качества классификации рассмотрим несколько способов при тренировке языковых моделей. 1) Сравниваться будут модели,используемые при создании эмбеддингов. Мы рассмтрим модель Bag of words, так как ее подход (незначимость порядка слов в контексте при использованиии формулы Байеса) идейно схож с созданной нами функцией, так как значимость слова для отдельного документа в подсчете весов не учитывается - учитывается только генеральная частотность. В этом отличие данной модели от метрики TF-IDF, которая учитывает встречаемость слова по корпусу, но и в отдельно рассматриваемом тексте. Особенность заключается в том, что вес становится тем больше, чем реже слово встречается в корпусе и чем чаще оно встречается в документе. 2) Сравниваться будут различные классификаторы, одним из которых будет логистическая регрессия, а другим - машина опорных векторов (SVM). Отличие между ними заключается в большей чувствительности SVM к выбросам, также логистическая регрессия производит вероятностные значения, в то время как SVM выдает 1 или 0 (в нашем случае желательно знать не апостериорную вероятность, а четкое значение класса).

In [204]:
df['texts_p'] = reviews_preprocessed
df = df.loc[df['tonality'] != 0.5]

In [208]:
X, y = df['texts_p'], df['tonality']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=4)
bow_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()
#TF-IDF
train_tfidf = tfidf_vectorizer.fit_transform(X_train)
test_tfidf = tfidf_vectorizer.transform(X_test)

#Vectorizer (Bag of Words Model)
train_bow = bow_vectorizer.fit_transform(X_train)
test_bow = bow_vectorizer.transform(X_test)

In [210]:
model1 = LogisticRegression(solver = 'liblinear')
model2 = SVC()

In [211]:
model1.fit(train_bow, y_train)
predictions1 = model1.predict(test_bow)

In [214]:
print(classification_report(y_test,predictions1))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94        35
         1.0       0.99      0.99      0.99       270

    accuracy                           0.99       305
   macro avg       0.97      0.97      0.97       305
weighted avg       0.99      0.99      0.99       305



In [216]:
model1.fit(train_tfidf, y_train)
predictions2 = model1.predict(test_tfidf)
print(classification_report(y_test,predictions2))

              precision    recall  f1-score   support

         0.0       1.00      0.06      0.11        35
         1.0       0.89      1.00      0.94       270

    accuracy                           0.89       305
   macro avg       0.95      0.53      0.53       305
weighted avg       0.90      0.89      0.85       305



In [217]:
model2.fit(train_bow, y_train)
predictions3 = model2.predict(test_bow)
print(classification_report(y_test,predictions3))

              precision    recall  f1-score   support

         0.0       0.94      0.89      0.91        35
         1.0       0.99      0.99      0.99       270

    accuracy                           0.98       305
   macro avg       0.96      0.94      0.95       305
weighted avg       0.98      0.98      0.98       305



In [218]:
model2.fit(train_tfidf, y_train)
predictions4 = model2.predict(test_tfidf)
print(classification_report(y_test,predictions4))

              precision    recall  f1-score   support

         0.0       1.00      0.77      0.87        35
         1.0       0.97      1.00      0.99       270

    accuracy                           0.97       305
   macro avg       0.99      0.89      0.93       305
weighted avg       0.97      0.97      0.97       305

